In [ ]:
def statistics(ws):
    import numpy as np

    weights = np.array(ws)
    #log returns:
    rets = np.log(data / data.shift(1))
    # expected portfolio return:
    pret = np.sum(rets.mean() * weights) * 252 
    # expected portfolio volatility:
    pvol = np.sqrt(np.dot(weights.T, np.dot(rets.cov() * 252, weights))) 
    # for simplicity we assume risk free short rate = 0:
    rf = 0
    # expected excess of the portfolio / expected standard deviation of the portfolio
    sharpe_ratio = (pret - rf) / pvol
    return np.array([pret, pvol, pret / pvol, sharpe_ratio])

In [ ]:
def min_func_sharpe(weights):
    return -statistics(weights)[2]

In [ ]:
def optimize_sharpe_ratio(initial_weights, bnds, cons):
    import scipy.optimize as sco
    
    opts = sco.minimize(min_func_sharpe, initial_weights, method = 'SLSQP', bounds = bnds, constraints = cons)
    return opts['x'].round(3)

In [ ]:
def load_data(symbols):
    import pandas as pd
    from dao import InvestDao
    
    dao = InvestDao()
    data = pd.DataFrame()
    for sym in symbols:
        rows = dao.sql("SELECT close FROM stock_quotes WHERE dtyymmdd > '2016-06-01' AND ticker = '%s' ORDER BY dtyymmdd" % sym)
        data[sym] = [float(r[0]) for r in rows]
    data.columns = symbols
    return data

In [ ]:
symbols = ['KGHM', 'PGNIG', 'PKOBP']
data = load_data(symbols)
noa = len(symbols)
initial_weights = noa * [1. / noa,]
# weights boundaries between 0 and 1
bnds = tuple((0, 1) for x in range(noa))
# constraint that all weights add up to 1
import numpy as np
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1 })

In [ ]:
optimal = optimize_sharpe_ratio(initial_weights, bnds, cons)
optimal_stats = statistics(optimal)

In [ ]:
print("The optimal portfolio weights are: %s" % optimal)
print("The optimal portfolio expected return is: %s, expected volatility: %s, Sharpe ratio: %s" % (optimal_stats[0], optimal_stats[1], optimal_stats[2]))